In [1]:
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
import pickle
import os

# Creating a Poker evaluation algorithm 

## Cactus Kev's Algorithm
This algorithm matches all hands to specific pre-computed value representing the rank among all unique rankings in a poker hand. 
```
+--------+--------+--------+--------+
|xxxbbbbb|bbbbbbbb|cdhsrrrr|xxpppppp|
+--------+--------+--------+--------+
```
where:
* b is bit turned on depending on rank
* cdhs is suit ie clubs, diamonds, heart, spades
* r is rank represented eg deuce=0, trey=1, four=2, five=3... Ace=12
* p is prime number corresponding to rank deuce = (deuce=2,trey=3,four=5,...,ace=41)

In [2]:
path = 'pokerevaluator/arrays/'

In [3]:
flushes = np.load(path + 'flushes.npy')
primes = np.load(path + 'primes.npy')
unique5 = np.load(path + 'unique5.npy')
with open(path + 'prodVal.pkl', 'rb') as f:
    prodVal = pickle.load(f)
with open(path + 'handDict.pkl', 'rb') as f:
    handDict = pickle.load(f)


In [79]:
def as_string(val: int):
    b_as_string = ("0" * (32 - len(bin(val)) )) + bin(val)[2:]
    return b_as_string

In [82]:
'''ranks = '23456789TJQKA'
suits = 'shdc'
handDict = {}
test = 0
for i, rank in enumerate(ranks):
    b = 1 << 16 + i
    r = i << 8
    p = primes[i]
    for j, suit in enumerate(suits):
        s = 2 ** j << 12
        binrep = b | r | p | s
        b_as_string = as_string(binrep)
        print(f"binary representation {b_as_string} length: {rank + suit}")
        handDict[rank + suit] = binrep
    #print(f"binary representation {bin(b)[2:]} length: {len(bin(b)[2:])}")
with open(path + 'handDict.pkl', 'wb') as f:
    pickle.dump(handDict, f, protocol=pickle.HIGHEST_PROTOCOL)'''

binary representation 000000000000010001000000000010 length: 2s
binary representation 000000000000010010000000000010 length: 2h
binary representation 000000000000010100000000000010 length: 2d
binary representation 000000000000011000000000000010 length: 2c
binary representation 000000000000100001000100000011 length: 3s
binary representation 000000000000100010000100000011 length: 3h
binary representation 000000000000100100000100000011 length: 3d
binary representation 000000000000101000000100000011 length: 3c
binary representation 000000000001000001001000000101 length: 4s
binary representation 000000000001000010001000000101 length: 4h
binary representation 000000000001000100001000000101 length: 4d
binary representation 000000000001001000001000000101 length: 4c
binary representation 000000000010000001001100000111 length: 5s
binary representation 000000000010000010001100000111 length: 5h
binary representation 000000000010000100001100000111 length: 5d
binary representation 000000000010001000

In [8]:
ranks = '23456789TJQKA'
suits = 'shdc'
deck = np.array([rank + suit for rank in ranks for suit in suits])
deck

array(['2s', '2h', '2d', '2c', '3s', '3h', '3d', '3c', '4s', '4h', '4d',
       '4c', '5s', '5h', '5d', '5c', '6s', '6h', '6d', '6c', '7s', '7h',
       '7d', '7c', '8s', '8h', '8d', '8c', '9s', '9h', '9d', '9c', 'Ts',
       'Th', 'Td', 'Tc', 'Js', 'Jh', 'Jd', 'Jc', 'Qs', 'Qh', 'Qd', 'Qc',
       'Ks', 'Kh', 'Kd', 'Kc', 'As', 'Ah', 'Ad', 'Ac'], dtype='<U2')

In [17]:
def evaluate(hand: list) -> int:
    bit_hand = [handDict[card] for card in hand]
    flushval = 0xF000
    flushval = (bit_hand[0] & bit_hand[1] & bit_hand[2] & bit_hand[3] & bit_hand[4]) & 0xF000
    if flushval > 0:
        lookupid = (bit_hand[0] | bit_hand[1] | bit_hand[2] | bit_hand[3] | bit_hand[4]) >> 16
        return flushes[lookupid]
    lookupid = (bit_hand[0] | bit_hand[1] | bit_hand[2] | bit_hand[3] | bit_hand[4]) >> 16
    straight_highcard = unique5[lookupid]
    if straight_highcard > 0:
        return straight_highcard
    lookupid = (bit_hand[0] & 0xFF) * (bit_hand[1] & 0xFF) * (bit_hand[2] & 0xFF) * (bit_hand[3] & 0xFF) * (bit_hand[4] & 0xFF)
    hand_rank = prodVal[lookupid]
    return hand_rank
print(evaluate(['2h', '3h','4h', '5h', '7h']))
print(evaluate(['2h', '3h','4h', '6h', '5h']))
print(evaluate(['As', '2s','3s', '4s', '5s']))
print(evaluate(['As', '2d','3h', '4c', '5d'])) 
print(evaluate(['As', 'Ad','Qs', 'Js', 'Ks']))
def evaluate_bithand(bit_hand: list) -> int:
    flushval = 0xF000
    flushval = (bit_hand[0] & bit_hand[1] & bit_hand[2] & bit_hand[3] & bit_hand[4]) & 0xF000
    if flushval > 0:
        lookupid = (bit_hand[0] | bit_hand[1] | bit_hand[2] | bit_hand[3] | bit_hand[4]) >> 16
        return flushes[lookupid]
    lookupid = (bit_hand[0] | bit_hand[1] | bit_hand[2] | bit_hand[3] | bit_hand[4]) >> 16
    straight_highcard = unique5[lookupid]
    if straight_highcard > 0:
        return straight_highcard
    lookupid = (bit_hand[0] & 0xFF) * (bit_hand[1] & 0xFF) * (bit_hand[2] & 0xFF) * (bit_hand[3] & 0xFF) * (bit_hand[4] & 0xFF)
    hand_rank = prodVal[lookupid]
    return hand_rank
def pick_winner(hands: list) -> int:
    '''returns index of hands'''
    hand_ranks = np.array([evaluate(hand) for hand in hands])
    return np.argsort(hand_ranks)[0] # 

1599
9
10
1609
3326


In [10]:
# testing
deck = np.array([rank + suit for rank in ranks for suit in suits]) # create a deck
def test(num_players, deck):
    # generate cards for each player
    np.random.shuffle(deck)
    hands = [deck[i*5:i*5+5] for i in range(num_players)]
    winner = pick_winner(hands)
    #for i, hand in enumerate(hands):
    #    print(f"Player {i+1}: {hand}")
    #print(f"Winner is Player {winner + 1} with {hands[winner]}")
for _ in range(5):
    print("="*30)
    test(5, deck)
    print("="*30)

In [32]:
# time test
import time
deck_bit = np.array([handDict[card] for card in deck])
iterations = 1_000_000
start = time.perf_counter()
for i in range(iterations):
    np.random.shuffle(deck_bit)
    evaluate_bithand(deck_bit[:5])
end = time.perf_counter()
print(f"Time taken: {end - start}")
print(f"evaluations/sec = {iterations/(end-start)}")

Time taken: 8.105540013999416
evaluations/sec = 123372.40927475014


In [154]:
from itertools import combinations
def texas_holdem(hands, shared_cards):
    combos = [np.array(combo) for combo in combinations(range(7), 5)]
    best_hands = []
    for hand in hands:
        sev_hand = np.concatenate((hand, shared_cards))
        best_hands.append(np.min([evaluate(sev_hand[combo]) for combo in combos]))
    return np.argsort(best_hands)[0]

def holdem_test(num_players):
    np.random.shuffle(deck)
    hands = [deck[i*2:i*2+2] for i in range(num_players)]
    shared_cards = deck[num_players * 2:num_players * 2 + 5]
    #for i, hand in enumerate(hands):
        #print(f"Player {i + 1}: {hand}")
    #print(f"Shared cards: {shared_cards}")
    winner = texas_holdem(hands, shared_cards)
    #print(f"Winner is Player {texas_holdem(hands, shared_cards) + 1} with hand {hands[winner]}")
iterations = 10000
num_players = 3
start = time.perf_counter()
for i in range(iterations):
    holdem_test(1)
end = time.perf_counter()
print(f"Evaluated {iterations} games in {end - start}")
print(f"Games/sec = {iterations/(end-start)}")

Evaluated 10000 games in 2.2091597309918143
Games/sec = 4526.607949489667
